## Setup & Importing packages 

In [4]:
# Importing required libraries

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from dotenv import load_dotenv
import langchain
import pinecone
import openai
import os

In [ ]:
# Importing the required keys from our .env file
load_dotenv()

## Reading the PDF
Using the PyPDFDirectoryLoader

In [6]:
# Defining the function to read the PDF from our directory using the PyPDFDirectoryLoader package
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [7]:
doc=read_doc('documents/')
len(doc)

32

## Dividing the PDF into Text Chunks
Using the RecursiveCharacterTextSplitter

In [ ]:
## Now, we will divide the pdfs that we read into text chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs